# 🌊 PINNs-MVP 完整專案指南
## Physics-Informed Neural Networks for Sparse-Data Turbulent Flow Reconstruction

---

### 📚 本 Notebook 內容概覽

#### Part 1: 專案介紹與環境設定
- 1.1 專案目標與核心技術
- 1.2 環境檢查與依賴安裝
- 1.3 專案結構導覽

#### Part 2: 資料準備與探索
- 2.1 JHTDB 資料下載與載入
- 2.2 感測點選擇策略（QR-Pivot）
- 2.3 資料視覺化與統計分析

#### Part 3: 模型架構深入解析
- 3.1 Fourier Features 原理
- 3.2 VS-PINN 變數尺度化
- 3.3 網路架構設計

#### Part 4: 訓練策略與技巧
- 4.1 快速基線訓練（100 epochs）
- 4.2 Reynolds 數遞增 Curriculum
- 4.3 GradNorm 動態權重平衡
- 4.4 訓練監控與診斷

#### Part 5: 評估與視覺化
- 5.1 物理一致性驗證
- 5.2 場重建誤差分析
- 5.3 湍流統計量對比
- 5.4 壁面剪應力評估

#### Part 6: 進階實驗
- 6.1 K-scan 實驗（最少感測點數）
- 6.2 Ablation Study（消融實驗）
- 6.3 Ensemble 不確定性量化

#### Part 7: 故障排除與最佳實踐
- 7.1 常見問題診斷
- 7.2 性能優化建議
- 7.3 檢查點管理

---

### 🎯 學習目標

完成本 Notebook 後，您將能夠：
- ✅ 理解 PINNs 在稀疏資料流場重建中的應用
- ✅ 掌握 Fourier Features 與 VS-PINN 的核心原理
- ✅ 執行完整的訓練-評估-診斷流程
- ✅ 分析並優化模型性能
- ✅ 進行可重現的科研實驗

---

### ⚙️ 運行環境建議

| 環境 | 推薦配置 | 預期時間 |
|------|---------|----------|
| **本地 GPU** | RTX 3090 / A6000 | 2-4 小時（完整訓練） |
| **Google Colab Pro** | A100 (40GB) | 1-2 小時 |
| **本地 CPU** | 16 核心以上 | 12-24 小時 |

---

**最後更新**: 2025-10-19  
**版本**: v2.0  
**作者**: PINNs-MVP Team

---

## ⚡ 快速開始指南

### 新用戶（第一次使用）

1. **環境設定** → 執行 Part 1.2 的所有 Cell
2. **資料準備** → 檢查 Part 2.1 資料是否存在
3. **快速訓練** → 使用 Part 4.1 的基線配置（10 分鐘）
4. **結果評估** → 參考 Part 5.1-5.3

### 進階用戶

- **Curriculum Learning**: Part 4.2（2-4 小時，最佳性能）
- **GradNorm 調優**: Part 4.3（關鍵參數 alpha=1.5）
- **Ablation Study**: Part 6.2（量化組件貢獻）
- **故障診斷**: Part 7.1（常見問題解決）

### 重要提醒

⚠️ **訓練建議使用命令行而非 Notebook**:
```bash
python scripts/train.py --cfg configs/my_exp.yml
```

Notebook 主要用於：理解原理、視覺化、快速驗證、結果分析

---

In [ ]:
# 快速開始訓練（取消註解後執行）
# !python scripts/train.py --cfg configs/my_exp.yml


---

# Part 1: 專案介紹與環境設定

---

## 1.1 專案目標與核心技術

### 🎯 研究問題

**挑戰**: 如何從極稀疏的感測點資料（K ≤ 50-500 點）重建完整的 3D 湍流場？

**傳統方法的限制**:
- 插值方法：無法捕捉湍流小尺度結構
- 資料同化：需要背景場且計算昂貴
- 純數據驅動：稀疏資料下嚴重過擬合

**我們的解決方案**: **Physics-Informed Neural Networks (PINNs)**

### 🔬 核心技術棧

#### 1. **Fourier Features (σ=2-5)**
```python
# 強化高頻結構捕捉能力
z = 2π * x @ B  # B ~ N(0, σ²)
φ(x) = [cos(z), sin(z)]
```
**作用**: 捕捉壁面邊界層高頻梯度（du/dy|_wall ≈ 1000）

#### 2. **VS-PINN 變數尺度化**
```python
# 針對各向異性流場優化
(X, Y, Z) = (N_x·x, N_y·y, N_z·z)
# 壁法向 N_y=12.0 > 流向 N_x=2.0
```
**作用**: 平衡各方向的梯度尺度，穩定訓練

#### 3. **GradNorm 動態權重**
```python
# 自動平衡多損失項
λ_i ← λ_i * (G_i / G_avg)^(-alpha)
# alpha=1.5, 每 1000 步更新
```
**作用**: 確保 PDE Loss Ratio ≥ 30%

#### 4. **Reynolds 數遞增 Curriculum**
```python
# 從低 Re_tau 逐步到高 Re_tau
Re_tau: 500 → 750 → 1000
```
**作用**: 避免高 Re 直接訓練的數值不穩定

### 🗂️ 資料集

**Johns Hopkins Turbulence Database (JHTDB)**
- **流場**: Channel Flow Re_τ=1000
- **解析度**: 2048×512×1536 (wavemodes)
- **域大小**: (L_x, L_y, L_z) = (8πh × 2h × 3πh), h=1
- **變數**: [u, v, w, p] (速度 + 壓力)

### 📖 參考文獻

1. Tancik et al. (2020) - *Fourier Features Let Networks Learn High Frequency Functions*
2. Chen et al. (2018) - *GradNorm: Gradient Normalization for Adaptive Loss Balancing*
3. VS-PINN (arXiv:2308.08468) - *Variable-Scaling Physics-Informed Neural Networks*
4. Raissi et al. (2019) - *Physics-informed neural networks: A deep learning framework*

## 1.2 環境檢查與依賴安裝

In [ ]:
# 檢查 Python 版本
import sys
print(f"Python 版本: {sys.version}")
assert sys.version_info >= (3, 10), "❌ 需要 Python 3.10 或更高版本"
print("✅ Python 版本符合需求")

In [ ]:
# 檢查 PyTorch 與 GPU
import torch
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU 設備: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("使用 Apple Silicon MPS")
    device = torch.device('mps')
else:
    print("⚠️ 僅 CPU 可用，訓練速度會較慢")
    device = torch.device('cpu')

print(f"\n✅ 使用設備: {device}")

In [ ]:
# 安裝依賴（如需要）
# 取消註釋以下行來安裝缺失的套件

# !pip install pyyaml h5py scipy matplotlib seaborn tensorboard
# !pip install netCDF4  # 用於 RANS 低保真場（可選）

# 驗證關鍵套件
import yaml
import numpy as np
import matplotlib.pyplot as plt
import h5py

print("✅ 所有關鍵依賴已安裝")
print(f"NumPy 版本: {np.__version__}")
print(f"Matplotlib 版本: {plt.matplotlib.__version__}")

### 在 Google Colab 掛載 Google Drive

若在 Colab 執行 Notebook，請先掛載 Drive 以讀寫 `pinns-mvp` 專案與資料。

In [ ]:
# 在 Google Colab 掛載 Google Drive 並切換到專案資料夾
try:
    from google.colab import drive  # type: ignore
except ModuleNotFoundError:
    print("Google Colab 未安裝，略過掛載步驟。")
else:
    drive.mount('/content/drive', force_remount=False)
    %cd /content/drive/MyDrive/pinns-mvp


## 1.3 專案結構導覽

In [ ]:
# 檢查專案結構
import os
from pathlib import Path

# 如果在 Colab，需要先切換到專案目錄
# %cd /content/pinns-mvp

project_root = Path.cwd()
print(f"專案根目錄: {project_root}\n")

# 檢查關鍵目錄
key_dirs = {
    'pinnx': '核心框架（模型、物理、訓練）',
    'configs': '配置文件範例',
    'scripts': '訓練與評估腳本',
    'tests': '單元測試',
    'data': '資料目錄（JHTDB 感測點）',
    'docs': '文檔與指南',
}

for dir_name, desc in key_dirs.items():
    dir_path = project_root / dir_name
    status = '✅' if dir_path.exists() else '❌'
    print(f"{status} {dir_name:15s} - {desc}")

# 列出模板配置
template_dir = project_root / 'configs' / 'templates'
if template_dir.exists():
    print(f"\n📁 可用配置模板:")
    for yml_file in sorted(template_dir.glob('*.yml')):
        print(f"   - {yml_file.name}")

### 專案架構說明

```
pinns-mvp/
├── pinnx/                    # 核心框架
│   ├── models/              # FourierMLP, SIREN, Wrappers
│   ├── physics/             # NS Equations, VS-PINN, RANS
│   ├── losses/              # GradNorm, Residuals, Priors
│   ├── sensors/             # QR-Pivot, Stratified Sampling
│   ├── train/               # Trainer, Factory, Config Loader
│   └── utils/               # Normalization, Denormalization
│
├── configs/
│   ├── templates/           # 標準化配置範例
│   │   ├── 2d_quick_baseline.yml
│   │   ├── 2d_medium_ablation.yml
│   │   ├── 3d_slab_curriculum.yml
│   │   └── 3d_full_production.yml
│   └── main.yml             # 主配置文件
│
├── scripts/
│   ├── train.py             # 主訓練腳本
│   ├── evaluate_checkpoint.py
│   └── comprehensive_evaluation.py
│
├── data/jhtdb/              # JHTDB 資料
│   ├── sensors_K500.npz     # QR-Pivot 感測點
│   └── channel_flow_re1000/ # 完整流場資料
│
└── docs/                    # 專案文檔
```

---

# Part 2: 資料準備與探索

---

## 2.1 JHTDB 資料下載與載入

### 選項 A: 使用預生成的感測點資料

In [ ]:
# 載入預生成的 K=500 感測點資料
data_path = 'data/jhtdb/sensors_K500.npz'

if os.path.exists(data_path):
    data = np.load(data_path)
    
    print("✅ 資料載入成功\n")
    print(f"感測點數量: {data['K']}")
    print(f"座標形狀: {data['sensor_coords'].shape}")
    print(f"場值形狀: {data['sensor_values'].shape}")
    print(f"\n變數順序: {list(data['variable_names'])}")
    print(f"\n域邊界:")
    print(f"  x ∈ [{data['domain_bounds']['x'][0]:.2f}, {data['domain_bounds']['x'][1]:.2f}]")
    print(f"  y ∈ [{data['domain_bounds']['y'][0]:.2f}, {data['domain_bounds']['y'][1]:.2f}]")
    print(f"  z ∈ [{data['domain_bounds']['z'][0]:.2f}, {data['domain_bounds']['z'][1]:.2f}]")
    
    # 提取資料
    sensor_coords = data['sensor_coords']
    sensor_values = data['sensor_values']
    K = int(data['K'])
    
else:
    print(f"❌ 資料文件不存在: {data_path}")
    print("請參考 Part 2.1 選項 B 或 C 生成資料")

### 選項 B: 從 JHTDB API 動態下載

In [ ]:
# 從 JHTDB 下載資料（需要網路連接與授權）
# 取消註釋以下代碼以執行

# from pinnx.dataio.jhtdb_client import create_jhtdb_manager
# from pinnx.sensors.qr_pivot import qr_pivot_sensor_selection

# # 設定參數
# K = 50  # 感測點數量
# time = 0.0  # 時間點

# # 創建 JHTDB 管理器
# manager = create_jhtdb_manager()

# # 定義感測點位置（均勻分佈）
# x = np.linspace(0, 25.13, 50)
# y = np.linspace(-1.0, 1.0, 20)
# z = np.linspace(0, 9.42, 30)
# X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
# all_coords = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

# # QR-Pivot 選擇最優感測點
# sensor_indices = qr_pivot_sensor_selection(all_coords, K=K)
# sensor_coords = all_coords[sensor_indices]

# # 從 JHTDB 獲取場值
# velocity = manager.get_velocity(sensor_coords[:, 0], sensor_coords[:, 1], sensor_coords[:, 2], time)
# pressure = manager.get_pressure(sensor_coords[:, 0], sensor_coords[:, 1], sensor_coords[:, 2], time)

# sensor_values = np.concatenate([velocity, pressure.reshape(-1, 1)], axis=1)
# print(f"✅ 從 JHTDB 下載 {K} 個感測點資料完成")

## 2.2 感測點選擇策略（QR-Pivot）

### QR-Pivot 原理

**目標**: 從 N 個候選點中選擇 K 個最具資訊量的感測點

**方法**: QR 分解的列主元策略
```
1. 構建觀測矩陣 Φ ∈ R^{N×M}（Fourier 特徵）
2. QR 分解: Φ @ P = Q @ R（P 為置換矩陣）
3. 選擇前 K 個列主元對應的座標
```

**優勢**:
- ✅ 最大化觀測矩陣的條件數（降低重建誤差）
- ✅ 保證空間覆蓋率（避免聚集）
- ✅ 計算效率高（O(NM²)）

In [ ]:
# 視覺化 QR-Pivot 選擇的感測點分佈
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 5))

# 3D 分佈
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(sensor_coords[:, 0], sensor_coords[:, 1], sensor_coords[:, 2], 
            c=sensor_values[:, 0], cmap='RdBu_r', s=50, alpha=0.6)
ax1.set_xlabel('x (streamwise)')
ax1.set_ylabel('y (wall-normal)')
ax1.set_zlabel('z (spanwise)')
ax1.set_title(f'QR-Pivot 感測點分佈 (K={K})')

# x-y 平面投影
ax2 = fig.add_subplot(132)
sc = ax2.scatter(sensor_coords[:, 0], sensor_coords[:, 1], 
                 c=sensor_values[:, 0], cmap='RdBu_r', s=100, alpha=0.7, edgecolors='k')
ax2.set_xlabel('x (streamwise)')
ax2.set_ylabel('y (wall-normal)')
ax2.set_title('x-y 平面投影')
ax2.axhline(y=-1.0, color='r', linestyle='--', label='壁面')
ax2.axhline(y=1.0, color='r', linestyle='--')
ax2.legend()
plt.colorbar(sc, ax=ax2, label='u 速度')

# y 方向分佈直方圖
ax3 = fig.add_subplot(133)
ax3.hist(sensor_coords[:, 1], bins=20, alpha=0.7, edgecolor='k')
ax3.set_xlabel('y (wall-normal)')
ax3.set_ylabel('感測點數量')
ax3.set_title('壁法向分佈')
ax3.axvline(x=-1.0, color='r', linestyle='--', label='壁面')
ax3.axvline(x=1.0, color='r', linestyle='--')
ax3.axvline(x=0.0, color='g', linestyle='--', label='中心線')
ax3.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 壁法向分佈統計:")
print(f"  壁面附近 (|y| > 0.8): {np.sum(np.abs(sensor_coords[:, 1]) > 0.8)} 點 ({np.sum(np.abs(sensor_coords[:, 1]) > 0.8)/K*100:.1f}%)")
print(f"  中心區域 (|y| < 0.5): {np.sum(np.abs(sensor_coords[:, 1]) < 0.5)} 點 ({np.sum(np.abs(sensor_coords[:, 1]) < 0.5)/K*100:.1f}%)")

## 2.3 資料視覺化與統計分析

In [ ]:
# 統計分析
import pandas as pd

# 創建 DataFrame
df = pd.DataFrame({
    'x': sensor_coords[:, 0],
    'y': sensor_coords[:, 1],
    'z': sensor_coords[:, 2],
    'u': sensor_values[:, 0],
    'v': sensor_values[:, 1],
    'w': sensor_values[:, 2],
    'p': sensor_values[:, 3]
})

print("📊 場變量統計摘要:\n")
print(df[['u', 'v', 'w', 'p']].describe())

# 視覺化分佈
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, var in enumerate(['u', 'v', 'w', 'p']):
    ax = axes[idx // 2, idx % 2]
    
    # 直方圖 + KDE
    ax.hist(df[var], bins=30, alpha=0.6, density=True, edgecolor='k', label='直方圖')
    
    # 統計資訊
    mean = df[var].mean()
    std = df[var].std()
    ax.axvline(mean, color='r', linestyle='--', linewidth=2, label=f'均值={mean:.3f}')
    ax.axvline(mean + std, color='orange', linestyle=':', label=f'±1σ')
    ax.axvline(mean - std, color='orange', linestyle=':')
    
    ax.set_xlabel(f'{var} 值')
    ax.set_ylabel('密度')
    ax.set_title(f'{var} 分佈 (mean={mean:.3f}, std={std:.3f})')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

# Part 3: 模型架構深入解析

---

## 5.2 場重建誤差分析

### 評估指標

1. **Relative L2 Error**:
   $$\text{L2}(f) = \frac{\|f_{\text{pred}} - f_{\text{true}}\|_2}{\|f_{\text{true}}\|_2}$$

2. **Point-wise RMSE**:
   $$\text{RMSE}(f) = \sqrt{\frac{1}{N}\sum_{i=1}^N (f_{\text{pred},i} - f_{\text{true},i})^2}$$

3. **Spectrum Error** (頻域分析):
   $$\text{Spectrum RMSE} = \frac{\|\hat{f}_{\text{pred}} - \hat{f}_{\text{true}}\|_2}{\|\hat{f}_{\text{true}}\|_2}$$

### 評估命令

```bash
# 評估檢查點
python scripts/evaluate_checkpoint.py \
    --checkpoint checkpoints/my_exp/best_model.pth \
    --config configs/my_exp.yml

# 綜合評估
python scripts/comprehensive_evaluation.py \
    --checkpoint checkpoints/my_exp/best_model.pth
```

In [ ]:
# 檢查點與綜合評估（取消註解後執行）
# !python scripts/evaluate_checkpoint.py --checkpoint checkpoints/my_exp/best_model.pth --config configs/my_exp.yml
# !python scripts/comprehensive_evaluation.py --checkpoint checkpoints/my_exp/best_model.pth


## 5.3 湍流統計量對比

### 關鍵統計量

1. **平均速度剖面** (Mean Velocity Profile):
   $$\langle u \rangle(y) = \frac{1}{L_x L_z} \int_0^{L_x} \int_0^{L_z} u(x,y,z) \, dx \, dz$$

2. **Reynolds 應力** (Reynolds Stress):
   $$\langle u'v' \rangle, \langle u'u' \rangle, \langle v'v' \rangle, \langle w'w' \rangle$$

3. **湍動能** (Turbulent Kinetic Energy):
   $$k = \frac{1}{2}(\langle u'^2 \rangle + \langle v'^2 \rangle + \langle w'^2 \rangle)$$

### 成功標準

- Mean profile correlation > 0.95
- Reynolds stress relative error < 30%
- TKE relative error < 40%

---

# Part 6: 進階實驗

---

## 6.1 K-scan 實驗（最少感測點數）

### 目標

找到維持重建精度的**最小感測點數** K_min。

### 實驗設計

測試 K ∈ {50, 100, 200, 500, 1000}:

```yaml
sensors:
  K: 50  # 修改此參數
  selection_method: "qr_pivot"
```

### 預期結果（基於文獻）

| K | Relative L2 | Wall τ Error | 備註 |
|---|-------------|--------------|------|
| 50 | ~50% | ~60% | 欠採樣 |
| 100 | ~35% | ~40% | 可接受 |
| 200 | ~28% | ~25% | 推薦 |
| 500 | ~22% | ~18% | 優良 |
| 1000 | ~18% | ~15% | 最佳 |

## 6.2 Ablation Study（消融實驗）

### 目的

量化各組件的貢獻：

1. **Fourier Features** (on/off)
2. **VS-PINN Scaling** (on/off)
3. **GradNorm** (on/off)
4. **Curriculum Learning** (single-stage vs multi-stage)

### 示例配置

```yaml
# Baseline: 無 Fourier
model:
  use_fourier: false

# vs

# Enhanced: 有 Fourier
model:
  use_fourier: true
  fourier_m: 64
  fourier_sigma: 3.0
```

### 預期改善

| 組件 | L2 Error 改善 | 原因 |
|------|--------------|------|
| Fourier Features | -15% | 高頻捕捉 |
| VS-PINN | -20% | 梯度平衡 |
| GradNorm | -25% | 多目標優化 |
| Curriculum | -30% | 穩定收斂 |

---

# Part 7: 故障排除與最佳實踐

---

## 7.1 常見問題診斷表

| 症狀 | 可能原因 | 解決方案 |
|------|---------|----------|
| Loss → NaN | 學習率過高 | 降低 lr 或啟用梯度裁剪 |
| PDE Ratio < 10% | GradNorm alpha 太小 | 提升至 1.5 |
| τ_w ≈ 0 | 壁面權重不足 | 增加 wall_constraint_weight |
| 過擬合感測點 | Data weight 過大 | 降低 data_weight 或增加 PDE weight |
| 訓練緩慢 | 批次大小太小 | 增加 batch_size (4096-8192) |
| 記憶體不足 | 模型太大或批次過大 | 減小 width/depth 或批次 |

## 7.2 性能優化建議

### 1. 硬體加速

```yaml
training:
  use_amp: true  # 自動混合精度（節省 40% 記憶體）
  device: "auto"  # 自動選擇 GPU/MPS
```

### 2. 數據並行（多 GPU）

```bash
# 使用 PyTorch DistributedDataParallel
python -m torch.distributed.launch --nproc_per_node=2 scripts/train.py --cfg configs/my_exp.yml
```

### 3. 梯度檢查點（節省記憶體）

```yaml
physics:
  use_gradient_checkpointing: false  # ⚠️ PINNs 不建議啟用
```

## 7.3 檢查點管理

### 保存策略

```yaml
logging:
  save_freq: 200  # 每 200 epoch 保存
  save_stage_checkpoints: true  # Curriculum 階段結束保存
```

### 恢復訓練

```bash
# 從檢查點恢復
python scripts/train.py --cfg configs/my_exp.yml --resume checkpoints/my_exp/epoch_800.pth
```

### 檢查點內容

- `model_state_dict`: 模型參數
- `optimizer_state_dict`: 優化器狀態
- `normalizer_stats`: 歸一化統計量
- `training_history`: 訓練歷史
- `config`: 完整配置

---

## ✅ 完成！

### 下一步建議

1. **快速驗證**: 運行 `configs/templates/2d_quick_baseline.yml` (10 分鐘)
2. **完整訓練**: 嘗試 `configs/templates/curriculum_reynolds_ramp.yml` (2-4 小時)
3. **進階實驗**: K-scan 或 Ablation Study
4. **論文複現**: 參考 `TECHNICAL_DOCUMENTATION.md`

### 相關資源

- **AGENTS.md**: 完整專案指南
- **docs/TRAINING_STABILIZATION_STRATEGIES.md**: 訓練策略詳解
- **docs/**: 所有技術文檔

---

**祝訓練順利！ 🚀**

## 4.4 訓練監控與診斷

### 關鍵指標

訓練過程中需要監控以下指標：

| 指標 | 目標值 | 診斷意義 |
|------|--------|----------|
| **PDE Loss Ratio** | **≥ 30%** | 物理約束執行強度 |
| **Wall Shear Stress** | **> 0.0005** | 壁面條件滿足（理論值 0.0025） |
| **Mass Conservation** | **< 1%** | 連續性方程誤差 |
| **L2 Error** | **< 30%** | 重建精度 |
| **Training Loss** | 穩定下降 | 收斂性 |

### 故障診斷流程

```bash
# 使用診斷工具
python scripts/debug/diagnose_piratenet_failure.py \
    --checkpoint checkpoints/my_exp/epoch_X.pth \
    --config configs/my_exp.yml \
    --output results/diagnosis/
```

**常見問題與解決方案**:

1. **PDE Loss Ratio < 10%**
   - **原因**: GradNorm alpha 太小
   - **解決**: 提升 alpha 至 1.5

2. **Wall Shear Stress ≈ 0**
   - **原因**: 壁面約束權重不足
   - **解決**: 增加 `wall_constraint_weight` 至 10-15

3. **Loss 變成 NaN**
   - **原因**: 學習率過高或梯度爆炸
   - **解決**: 啟用梯度裁剪 `gradient_clip_val: 1.0`

---

# Part 5: 評估與視覺化

---

## 5.1 物理一致性驗證

### 驗證項目

1. **質量守恆** (連續性方程):
   $$\nabla \cdot \mathbf{u} = \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} + \frac{\partial w}{\partial z} \approx 0$$
   
2. **動量守恆** (NS 方程殘差):
   $$\frac{\partial u}{\partial t} + \mathbf{u} \cdot \nabla u = -\frac{1}{\rho}\nabla p + \nu \nabla^2 u + f$$

3. **邊界條件**:
   - 壁面: $u = v = w = 0$ at $y = \pm 1$
   - 周期性: $u(x, y, 0) = u(x, y, 3\pi)$

4. **壁面剪應力**:
   $$\tau_w = \mu \frac{\partial u}{\partial y}\bigg|_{y=\pm1} \approx 0.0025$$

### 驗證工具

```bash
# 物理一致性驗證
pytest tests/test_physics.py
pytest tests/test_physics_validation.py

# 守恆律驗證
python scripts/validation/validate_ns_conservation.py
```

In [ ]:
# 訓練失敗診斷與物理驗證（取消註解後執行）
# !python scripts/debug/diagnose_piratenet_failure.py --checkpoint checkpoints/my_exp/epoch_X.pth --config configs/my_exp.yml --output results/diagnosis/

# !pytest tests/test_physics.py
# !pytest tests/test_physics_validation.py
# !python scripts/validation/validate_ns_conservation.py


## 4.3 GradNorm 動態權重平衡

### 問題：多目標優化的挑戰

PINNs 需要同時優化多個損失項：
- **Data Loss**: 感測點擬合
- **PDE Residuals**: 物理方程滿足
- **Boundary Conditions**: 邊界條件
- **Priors**: 先驗知識（可選）

**固定權重的問題**:
- 不同損失項尺度差異巨大（10^-5 到 10^2）
- 訓練過程中相對重要性會變化
- 容易陷入 trivial solution（過擬合感測點，忽略物理）

### GradNorm 解決方案

**核心思想**: 動態調整權重，使各損失項的梯度範數保持平衡

$$
\lambda_i(t+1) = \lambda_i(t) \cdot \left( \frac{\|\nabla_\theta L_i\|}{\bar{G}(t)} \right)^{-\alpha}
$$

其中：
- $\|\nabla_\theta L_i\|$: 損失項 $i$ 的梯度範數
- $\bar{G}(t)$: 所有損失項梯度範數的平均值
- $\alpha$: 調整強度（**關鍵參數**）

### 參數配置（基於 SYSTEM_COUPLING_ANALYSIS_LESSONS.md）

| 參數 | 推薦值 | 說明 |
|------|--------|------|
| **alpha** | **1.5** | 響應強度（0.12 太小 → PDE Ratio < 10%） |
| **update_freq** | **1000** | 更新頻率（避免計算開銷與震盪） |

**錯誤示範**（PirateNet 失敗案例）:
```yaml
# ❌ 不推薦
losses:
  grad_norm_alpha: 0.12  # 太小，無法有效平衡
  weight_update_freq: 50  # 過於頻繁
```

**正確配置**:
```yaml
# ✅ 推薦
losses:
  adaptive_weighting: true
  grad_norm_alpha: 1.5
  weight_update_freq: 1000
```

### 預期效果

- **PDE Loss Ratio**: 從 0.7% 提升至 ≥ 30%
- **壁面剪應力**: 從 ~0 恢復至理論值附近（τ_w ≈ 0.0025）
- **訓練穩定性**: 顯著改善，損失平滑下降

## 4.2 Reynolds 數遞增 Curriculum

### 動機

**問題**: 直接在高 Reynolds 數（Re_τ=1000）下訓練模型，容易因梯度過於剛性（stiff）而導致數值不穩定、收斂困難。

**解決方案**: 採用課程學習，從一個較簡單的物理場景（低 Re_τ）開始，讓模型學會基本的流動結構，然後逐步增加複雜性（提升 Re_τ），最終在目標 Reynolds 數下進行精煉。

### 新的 3 階段 Curriculum 策略

我們的新預設課程 (`3d_full_production.yml`) 包含三個階段：

| 階段 | Epochs | Re_tau | ν (黏度) | 學習率 | 主要目標 |
|---|---|---|---|---|---|
| **Stage 1** | 0-1500 | 500 | 1.0e-4 | 1.0e-3 | 穩定建立基本流場 |
| **Stage 2** | 1500-3500 | 750 | 6.67e-5 | 0.9e-3 | 過渡到更複雜的湍流結構 |
| **Stage 3** | 3500-5000 | 1000 | 5.0e-5 | 0.729e-3 | 在目標 Re 數下精煉細節 |

### 訓練命令

```bash
# 課程學習是新預設，直接執行生產模板即可
python scripts/train.py --cfg configs/templates/3d_full_production.yml
```

In [ ]:
# 課程學習訓練（取消註解後執行）
# !python scripts/train.py --cfg configs/templates/3d_full_production.yml


In [ ]:
# 快速基線訓練示例（如果在 Notebook 中直接訓練）
# 實際訓練建議使用命令行: python scripts/train.py --cfg configs/my_exp.yml

# 這裡展示如何透過配置創建訓練器
import yaml
from pathlib import Path

# 載入快速基線配置
config_path = Path('configs/templates/2d_quick_baseline.yml')

if config_path.exists():
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    print("⚙️ 快速基線配置摘要:\n")
    print(f"   實驗名稱: {config['experiment']['name']}")
    print(f"   訓練輪數: {config['training']['epochs']}")
    print(f"   學習率: {config['training']['optimizer']['lr']}")
    print(f"   批次大小: {config['training']['batch_size']}")
    print(f"\n   感測點數: K={config['sensors']['K']}")
    print(f"   選擇策略: {config['sensors']['selection_method']}")
    print(f"\n   模型寬度: {config['model']['width']}")
    print(f"   模型深度: {config['model']['depth']}")
    print(f"   Fourier σ: {config['model'].get('fourier_sigma', 'N/A')}")
    print(f"\n   預期時間: {config.get('expected_duration', '5-10 分鐘')}")
    
    print(f"\n📝 訓練命令:")
    print(f"   python scripts/train.py --cfg {config_path}")
    
else:
    print(f"❌ 配置文件不存在: {config_path}")
    print("請確認專案目錄結構正確")

---

# Part 4: 訓練策略與技巧

---

## 4.1 快速基線訓練（100 epochs，5-10 分鐘）

### 目標
- 驗證環境配置正確
- 快速測試感測點數據質量
- 建立性能基線

### 配置文件

使用 `configs/templates/2d_quick_baseline.yml` 作為起點。

In [ ]:
# 創建並檢查模型架構
from pinnx.models.fourier_mlp import create_pinn_model

# 配置：Enhanced 模式
model_config = {
    'type': 'enhanced_fourier_mlp',
    'in_dim': 3,
    'out_dim': 4,
    'width': 256,
    'depth': 6,
    'fourier_m': 64,
    'fourier_sigma': 3.0,
    'activation': 'swish',
    'use_fourier': True,
    'use_residual': True,
    'use_layer_norm': False,
    'dropout': 0.0
}

model = create_pinn_model(model_config)
model_summary = model.get_model_summary()

print("🏗️ 模型架構摘要:\n")
for key, value in model_summary.items():
    if key == 'total_params':
        print(f"   {key:20s}: {value:,}")
    else:
        print(f"   {key:20s}: {value}")

print(f"\n📐 模型結構詳細:")
print(model)

# 測試前向傳播
test_coords = torch.randn(10, 3)  # 10 個測試點
test_coords.requires_grad = True

with torch.no_grad():
    predictions = model(test_coords)

print(f"\n✅ 前向傳播測試:")
print(f"   輸入形狀: {test_coords.shape}")
print(f"   輸出形狀: {predictions.shape}")
print(f"   輸出範圍: [{predictions.min():.3f}, {predictions.max():.3f}]")

## 3.3 完整網路架構

### 網路流程圖

```
輸入座標 (x, y, z)
    ↓
[可選] VS-PINN 縮放: (X, Y, Z) = (N_x·x, N_y·y, N_z·z)
    ↓
Fourier Features 編碼: φ(X,Y,Z) ∈ R^{2m}
    ↓
[可選] 投影層: Linear(2m → width) + Swish
    ↓
隱藏層 × depth: 
  ├─ [RWF]Linear(width → width)
  ├─ [可選] LayerNorm
  ├─ Activation (tanh/swish/sine)
  ├─ [可選] Dropout
  └─ [可選] Residual Connection
    ↓
輸出層: Linear(width → 4)  # [u, v, w, p]
    ↓
輸出反歸一化（恢復物理尺度）
```

### 配置範例對比

| 配置 | Width | Depth | Fourier m | Activation | Residual | RWF | 參數量 | 適用場景 |
|------|-------|-------|-----------|------------|----------|-----|--------|---------|
| **Standard** | 128 | 4 | 32 | tanh | ❌ | ❌ | ~130K | 快速驗證 |
| **Enhanced** | 256 | 6 | 64 | swish | ✅ | ❌ | ~580K | 生產訓練 |
| **PirateNet** | 200 | 8 | 256 | sine | ✅ | ✅ | ~1.2M | 高精度研究 |

In [ ]:
# 視覺化 VS-PINN 縮放效果
from pinnx.physics.vs_pinn_channel_flow import VSPINNChannelFlow

# 創建 VS-PINN 物理模組
vs_pinn = VSPINNChannelFlow(
    scaling_factors={'N_x': 2.0, 'N_y': 12.0, 'N_z': 2.0},
    physics_params={'nu': 5e-5, 'dP_dx': 0.0025, 'rho': 1.0}
)

print("⚙️ VS-PINN 縮放係數:")
print(f"   N_x (流向): {vs_pinn.N_x.item():.1f}")
print(f"   N_y (壁法向): {vs_pinn.N_y.item():.1f}")
print(f"   N_z (展向): {vs_pinn.N_z.item():.1f}")

# 模擬縮放前後的梯度尺度
y_coords = np.linspace(-1, 1, 100)
du_dy_physical = 1000 * (1 - y_coords**2)  # 模擬壁法向梯度（簡化）

# 縮放後的座標
Y_scaled = vs_pinn.N_y.item() * y_coords

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左圖：縮放前
axes[0].plot(y_coords, du_dy_physical, 'b-', linewidth=2, label='∂u/∂y (物理空間)')
axes[0].set_xlabel('y (壁法向)', fontsize=12)
axes[0].set_ylabel('∂u/∂y', fontsize=12)
axes[0].set_title('縮放前：梯度尺度極端不平衡', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()
axes[0].axhline(y=0, color='k', linestyle='--', linewidth=0.5)
axes[0].text(0.05, 0.95, f'最大梯度: {du_dy_physical.max():.0f}', 
             transform=axes[0].transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

# 右圖：縮放後
du_dY_scaled = du_dy_physical / vs_pinn.N_y.item()  # 鏈式法則補償
axes[1].plot(Y_scaled, du_dY_scaled, 'r-', linewidth=2, label='∂u/∂Y (縮放空間)')
axes[1].set_xlabel('Y = N_y · y (縮放座標)', fontsize=12)
axes[1].set_ylabel('∂u/∂Y', fontsize=12)
axes[1].set_title('縮放後：梯度尺度平衡', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend()
axes[1].axhline(y=0, color='k', linestyle='--', linewidth=0.5)
axes[1].text(0.05, 0.95, f'最大梯度: {du_dY_scaled.max():.0f}', 
             transform=axes[1].transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\n✅ 縮放效果:")
print(f"   縮放前最大梯度: {du_dy_physical.max():.0f}")
print(f"   縮放後最大梯度: {du_dY_scaled.max():.0f} (降低 {vs_pinn.N_y.item():.0f}x)")
print(f"   梯度平衡比: {du_dy_physical.max() / du_dY_scaled.max():.1f} ≈ N_y")

## 3.2 VS-PINN 變數尺度化

### 動機

Channel Flow 是**各向異性流場**：
- 流向 (x): 特徵尺度 $L_x \sim 8\pi$
- 壁法向 (y): 特徵尺度 $\delta_\nu \sim 0.001$ (邊界層厚度)
- 展向 (z): 特徵尺度 $L_z \sim 3\pi$

**問題**: 標準 PINN 在各方向使用相同權重，導致：
- 壁法向梯度 ($\frac{\partial u}{\partial y}$) 數值遠大於流向梯度
- 訓練不穩定，損失函數難以平衡

### VS-PINN 解決方案

**座標變換**:
$$
(X, Y, Z) = (N_x \cdot x, N_y \cdot y, N_z \cdot z)
$$

**鏈式法則**:
$$
\frac{\partial u}{\partial x} = N_x \cdot \frac{\partial u}{\partial X}, \quad 
\nabla^2 u = N_x^2 \frac{\partial^2 u}{\partial X^2} + N_y^2 \frac{\partial^2 u}{\partial Y^2} + N_z^2 \frac{\partial^2 u}{\partial Z^2}
$$

**推薦值** (基於 VS-PINN 論文與實驗):
- $N_x = 2.0$ (流向，最柔軟)
- $N_y = 12.0$ (壁法向，最剛性)
- $N_z = 2.0$ (展向，與流向相似)

In [ ]:
# 視覺化 Fourier Features 的作用
import torch
from pinnx.models.fourier_mlp import FourierFeatures

# 創建不同 sigma 的 Fourier 編碼器
sigmas = [1.0, 3.0, 5.0]
colors = ['blue', 'orange', 'green']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1D 測試函數：高頻 sine 波
x_test = torch.linspace(-1, 1, 1000).unsqueeze(1)  # [1000, 1]
y_true = torch.sin(20 * np.pi * x_test)  # 高頻函數

for idx, (sigma, color) in enumerate(zip(sigmas, colors)):
    ax = axes[idx]
    
    # 創建 Fourier Features
    fourier = FourierFeatures(in_dim=1, m=32, sigma=sigma, multiscale=False)
    
    # 編碼
    with torch.no_grad():
        features = fourier(x_test)  # [1000, 64]
    
    # 可視化前 10 個特徵
    ax.plot(x_test.numpy(), features[:, :10].numpy(), alpha=0.3, linewidth=0.8)
    ax.set_title(f'Fourier Features (σ={sigma})', fontsize=14, fontweight='bold')
    ax.set_xlabel('x')
    ax.set_ylabel('Feature Value')
    ax.grid(alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
    
    # 統計資訊
    feature_freq = features.std(dim=0).mean().item()
    ax.text(0.05, 0.95, f'平均振幅: {feature_freq:.2f}', 
            transform=ax.transAxes, fontsize=10, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"\n💡 關鍵觀察:")
print(f"   σ=1.0: 低頻特徵，振幅較小，適合全局趨勢")
print(f"   σ=3.0: 中頻特徵，振幅適中，平衡全局與細節")
print(f"   σ=5.0: 高頻特徵，振幅較大，捕捉邊界層梯度")

## 3.1 Fourier Features 原理與視覺化

### 理論基礎

**問題**: 標準 MLP 難以捕捉高頻函數（Rahaman et al., 2019）

**解決方案**: Random Fourier Features (RFF)

$$
\phi(x) = [\cos(2\pi B x), \sin(2\pi B x)]
$$

其中 $B \in \mathbb{R}^{d \times m}$，$B_{ij} \sim \mathcal{N}(0, \sigma^2)$

**參數 $\sigma$ 的作用**:
- $\sigma$ 小（≈0.5-1.0）：捕捉低頻特徵（全局趨勢）
- $\sigma$ 大（≈5-10）：捕捉高頻特徵（邊界層梯度）

**Channel Flow 的選擇**: $\sigma = 2-5$（TRAINING_STABILIZATION_STRATEGIES.md）
- 理由：壁面剪應力梯度 $\frac{du}{dy}|_{\text{wall}} \approx 1000$
- 邊界層厚度：$\delta_\nu = \frac{1}{\text{Re}_\tau} \approx 0.001$